In [ ]:
import json

with open('training.json', 'r') as f:
    training_config = json.load(f)

In [ ]:
training_config

In [ ]:
import torch
torch.cuda.set_device(0)

device_idx = 3
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#device = torch.device("cpu")
print("Device:", device)
print("Current device idx:", torch.cuda.current_device())

In [ ]:
bs = training_config['batch_size']
n_bars = training_config['model']['n_bars']
nw = 10

ds_dir = "data/prova/"

In [ ]:
from torch.utils.data import Subset
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader
from data import PolyphemusDataset
import torch


tr_ratio = 0.7
vl_ratio = 0.1

dataset = PolyphemusDataset(ds_dir, n_bars)
ds_len = len(dataset)

print('Dataset len:', len(dataset))

train_len = int(tr_ratio * len(dataset)) 
valid_len = int(vl_ratio * len(dataset))
test_len = len(dataset) - train_len - valid_len
tr_set, vl_set, ts_set = random_split(dataset, (train_len, valid_len, test_len))

trainloader = DataLoader(tr_set, batch_size=bs, shuffle=True, num_workers=nw)
validloader = DataLoader(vl_set, batch_size=bs, shuffle=False, num_workers=nw)

tr_len = len(tr_set)
vl_len = len(vl_set)
ts_len = len(ts_set)

print('TR set len:', len(tr_set))
print('VL set len:', len(vl_set))
print('TS set len:', len(ts_set))

In [ ]:

from utils import print_params
import os
from model import VAE
import torch.optim as optim
from training import PolyphemusTrainer
from training import ExpDecayLRScheduler

print("Creating the model and moving it to the specified device...")

# Create model dir
models_dir = 'models_prova/'
model_name = 'prova'
model_dir = os.path.join(models_dir, model_name)
os.makedirs(models_dir, exist_ok=True) 
os.makedirs(model_dir, exist_ok=True)

# Creating the model
vae = VAE(**training_config['model'], device=device).to(device)

print_params(vae)
print()

# Creating optimizer and scheduler
optimizer = optim.Adam(vae.parameters(), **training_config['optimizer'])
scheduler = ExpDecayLRScheduler(
    optimizer=optimizer,
    **training_config['scheduler']
)

# Save config
config_path = os.path.join(model_dir, 'configuration')
torch.save(training_config, config_path)

print('--------------------------------------------------\n')

trainer = PolyphemusTrainer(
    model_dir,
    vae,
    optimizer,
    lr_scheduler=scheduler,
    save_every=1,
    print_every=1,
    iters_to_accumulate=1,
    device=device,
    **training_config['beta_annealing']
)
trainer.train(trainloader, validloader=None, epochs=100)